In [1]:
import json
import sys
sys.path.append('../../..')

In [2]:
import collections
import copy

In [10]:
import re
import pandas as pd

ans_pattern = re.compile(r'@content\d@')
def is_multi_answer(answer):
    docs = ans_pattern.findall(answer)
    return len(set([int(doc[-2:-1]) for doc in docs])) > 1

train_df = pd.read_csv('../input/original/train_round_0.csv')
train_df['is_multi_answer'] = train_df['answer'].map(is_multi_answer)

In [13]:
train_df[train_df['is_multi_answer'] == True]['question'].values.tolist()

['法国政界对法国总统的什么行为感到抵触？',
 '俄罗斯战机在太平洋侵犯哪个国家的领空？该国家海空防卫在亚洲范围内位居第几位？',
 '2003年3月，第一艘“可畏”号龙骨在哪国洛里昂造船厂安放，“可畏”级护卫舰舰使用6公斤炮弹，射程为多少公里？',
 '红旗2防空导弹是哪个国家SA-2“方针”地空导弹的中国版本，80年代时，伊朗通过第三国，获得了红旗2防空导弹，有击落哪个飞机的记录？',
 'WGS卫星采用什么电池阵列？这些卫星能为美军提供什么服务？',
 '非盟和平与安全理事会向布隆迪派遣5000人的军队是为了防止什么？这支军队主要由什么构成？',
 '“石鱼”水雷储存时间是多久？它的结构特点是什么？',
 '哪两个地方遭遇了“喀秋莎”火箭弹的袭击？',
 '旋风级巡逻艇首艇的服役时间是什么时候？目前有多少艘该级巡逻艇部署在巴林的美军基地？',
 '派拉蒙-掠夺者最高速度可以达到每小时多少公里？派拉蒙掠夺者动力配置来自哪里？',
 '世界上十分有名的牵引轻型野战火炮产自哪里？其射程为多少？',
 '福特号的标准排水量为多少万吨？按照设计标准，福特号先进武器升降机旨在以每分钟45米的速度将多少吨的弹药运到飞行甲板上？',
 '59式57毫米高射炮大多被装备于哪种单位？重量达到了多少千克？',
 '鹰击-62型反舰导弹的射程为多少？它有哪些特点？',
 '苏联列装的最后一款主战坦克也是第一辆安装燃气涡轮发动机的苏制主战坦克，该型坦克的最高时速是多少？且该型坦克的最新型号是什么？',
 '东风-5B采用了什么形式的战斗部？战斗部载荷为多少？',
 '图-22M最大起飞重量为多少吨？可以搭载几枚超音速导弹？',
 '1155型“无畏”级大型反潜舰有哪几艘?',
 '轰-6K可搭载多少枚核弹头战略、巡航导弹？ 解放军空军开发轰-6k的意图是什么？',
 '俄罗斯研发的Yu-71“战略高超声速助推-滑翔飞行器”曾用了哪些导弹进行了测试？',
 'T-72的车体和炮塔采用了什么的布置？捷克陆军共拥有多少辆T-72坦克？',
 '美国海军陆战队有多少架KC-130J加油机？该款加油机是使用哪款运输机作为平台改装的？',
 '苏-34战斗轰炸机在什么时候进入俄罗斯空军服役？它和什么型飞机一样保留了独特的前置鸭翼布局？',
 '俄罗斯国防部18日在官方公布什么战机在黑海和波罗的海

In [3]:
# from eval_metric import normalize, compute_bleu_rouge

In [3]:
question_dict = {}
with open('/home/lq/Research/Reading-Comprehension/les-military-mrc/input/mrc_dataset_question_split/test_r0.json') as f:
    for line in f:
        sample = json.loads(line)
        question_dict[sample['question_id']] = sample['question']

In [4]:
len(question_dict)

4969

In [5]:
cnt = 0
multi_ans_question_id_list = []
for id_, question in question_dict.items():
    if (question.count('?') + question.count('？')) > 1 or (question.count('多少') + question.count('哪') + question.count('什么') + question.count('谁')) > 1:
#         print(question)
        cnt += 1
        multi_ans_question_id_list.append(id_)
#     elif '分别' in question and '和' in question:
#         print(id_, question)
#     elif '哪两' in question:
#         print(id_, question)
    
print(cnt)

285


In [6]:
multi_ans_question_id_list += ['Q_06031cjhhh', 'Q_47760knsdc', 'Q_52500cnjsq', 'Q_36961chjcs', 'Q_22570hycqj',
                               'Q_40246dnjsn']

In [7]:
len(multi_ans_question_id_list)

291

In [8]:
with open('/home/lq/Research/Reading-Comprehension/yingzq/les-military-mrc/les_mrc/bert_baseline/models/les_bert_wwm_add_back_trans_trains/nbest_predictions_checkpoint_test.json') as f:
    all_nbest_json = json.loads(f.read())

In [9]:
all_samples = collections.defaultdict(list)
need_skippeed_list = [',', '.', '。']
for qas_id, nbest_json in all_nbest_json.items():
    text = ''
    prob = 0.0
    logit = 0.0
    for entry in nbest_json:
        if entry['text'].strip() and entry['text'] not in need_skippeed_list:
            text = entry['text'].strip()
            logit = entry['start_logit'] + entry['end_logit']
            prob = entry['probability']
            break
    all_samples[qas_id.split('##')[0]].append([text, logit, prob])
all_predictions = {}
multi_pred = []
cnt = 0
for question_id, sample in all_samples.items():
    sample = sorted(sample, key=lambda x: x[1], reverse=True)
    all_predictions[question_id] = sample[0][0]
#     print(sample[0][2])
    # 简单的多答案选择模块
    if question_id in multi_ans_question_id_list:
#         for i in range(1, 5):
#             ans1 = normalize([sample[0][0]])
#             ans2 = normalize([sample[i][0]])
#         if sample[0][0] == sample[1][0]:
#             continue
        all_predictions[question_id] = all_predictions[question_id] + '#' + sample[1][0]
        print(all_predictions[question_id])
        cnt += 1
#     if sample[1][0] != '' and sample[1][2] > 0.5:
#         # 有可能具有多答案
#         if sample[0][0] == sample[1][0]:
#             continue
#         if sample[1][0] in sample[0][0] or sample[0][0] in sample[1][0]:
#             continue
#         ans1 = normalize([sample[0][0]])
#         ans2 = normalize([sample[1][0]])
#         bleu_rouge = compute_bleu_rouge({'item': ans1}, {'item': ans2})
#         if bleu_rouge['Bleu-4'] > 0.2 or bleu_rouge['Rouge-L'] > 0.2:
#             continue
#         print('{} have multi-ans, take care of it'.format(question_id))
#         multi_pred.append(question_id)
#         cnt += 1
#         all_predictions[question_id] = all_predictions[question_id] + '#' + sample[1][0]
        
print(cnt)

5.49米#货舱宽度为5.49米
新加坡军队即将到达服役期限的M16突击步枪#885米
1951年开始研制#F-104战斗机。其实这种系统就是在F-104战斗机机腹下方安装了一台固体火箭发动机。
整车高宽比为0.91#这辆车的重量达到了5吨
12艘#两具
可能会配备现在正在开发的远程防区外武器(LRSO)#飞翼结构
射速快，后座力低#11.43毫米
瑞典萨博(SAAB)公司#美国波音公司
长约8米#南非桑达克·奥斯特瑞尔·比派克公司
采用了气动、气冷原理#采用气动、气冷原理
公路机动模式#卡车拖挂式运载
突破了大型舰艇总体设计、信息集成、总装建造等一系列关键技术#中国船舶重工集团701研究所设计
25辆#“豹”1主战坦克
MG42#800米以上
18公里#最大射程约11800米
MIM-23“霍克”地空导弹#184公里
西班牙内战#30年代
7吨#全新的航空复合材料
东风11导弹#40-50米水平
比利时小镇巴斯通#OH-58D型“基奥瓦”侦察直升机
R-73导弹#20-40千米之间
马丁公司#美国洛克希德·马丁公司
240艘#200余艘
誉21型发动机#190
布雷默顿预备役舰队#42架
SS-X-29#其发射车重量为80吨
1138毫米#射击稳定性好，攻击力强
主因为舰上搭载16枚杀伤力强大的雄风系列反舰导弹。#沱江级舰具匿踪能力，速度快特性，并能够携带8枚雄风二型与8枚雄风三型反舰导弹，火力强大。
280毫米#4级共17艘
采用了两根枪管#4.42千克
123架#3架美制C-130运输机
1000吨#装备有32单元垂发系统和威力强大的海红旗-16防空导弹，还装备有两组四联装鹰击-83反舰导弹
由莫斯科“劳动旗帜”工厂（现米格公司第二生产中心）#63吨
一个中国原则和中美三个联合公报规定#一个中国原则和中美三个联合公报
1984-1991年#"特拉法加"级攻击型核潜艇
1.78马赫#高强度、耐高温的BMI复合材料
约9800公里每小时#300～500公里
B-2隐身战略轰炸机#61米
20毫米#1541架
霹雳15远程空空导弹#2架
是25发一小节的分段式结构#1000米以内的集团目标
反潜和护航#134米
全枪重量为3.45千克#14颗
18艘#水上航速12节
具备浮渡精准射击能力#105毫米
伊拉克#“桑普森”号驱逐舰
40至50千米#“物美价廉”的平民路线

In [10]:
with open('multi_ans_bert_wwm_add_back_translate_trainsets.json', "w") as writer:
    writer.write(json.dumps(all_predictions, indent=4, ensure_ascii=False) + "\n")

In [11]:
all_samples['Q_21851qshyg']

[['1951年开始研制', 10.574229717254639, 0.5829507219310236],
 ['1951年', -0.39258645474910736, 1.1752164383714876e-06],
 ['16.66米', 1.2538080736994743, 8.580282433662477e-06],
 ['F-104战斗机。其实这种系统就是在F-104战斗机机腹下方安装了一台固体火箭发动机。',
  2.6591626405715942,
  0.063830777204302],
 ['1958年初', 0.6829356849193573, 1.77293278179766e-06]]

In [12]:
question_dict['Q_21851qshyg']

'F-104是什么时候开始研制的？F-104的零长起飞系统（ZELL）”具体是什么？'